In [1]:
# Colab setup code block
try:
    import google.colab
    print("Running on Google Colab. Setting up environment...")

    import os
    if not os.path.exists('CMAI-E91-Students'):
        !git clone https://github.com/algolab-quantique/CMAI-E91-Students.git

    os.chdir('CMAI-E91-Students/Part_3_Assignment')
    !pip install -r ../requirements.txt -q
    print("Environment setup complete!")
except ImportError:
    print("Not running on Google Colab. Skipping setup.")


Running on Google Colab. Setting up environment...
Environment setup complete!


# Coding Challenge - E91 with |Φ-⟩


In the workshop, we implemented E91 using the **singlet state** |Ψ-⟩ = (|01⟩ - |10⟩)/√2.

In this assignment, you will adapt E91 to use the **|Φ-⟩ Bell state** = (|00⟩ - |11⟩)/√2.

**Key difference:**
- |Ψ-⟩ gives **anti-correlated** results (Alice=0 → Bob=1)
- |Φ-⟩ gives **correlated** results (Alice=0 → Bob=0)


## What you will learn:

1. **Any Bell state works** for E91, but you need the right formula!
2. **Correlated vs Anti-correlated** states affect key extraction
3. **The CHSH formula changes** depending on the Bell state
4. **Experimental approach**: try all possibilities, find what gives 2√2

**TODO:** Adapt the E91 protocol to use the |Φ-⟩ Bell state and decrypt the secret messages.
- Task 1: Create the |Φ-⟩ circuit (hint: add ONE gate to |Φ+⟩)
- Task 2: Find the correct CHSH formula (try all 4 minus positions!)


### Setup

In [2]:
# ============================================================
# SETUP - All imports and constants
# ============================================================

import random
import numpy as np
from qiskit import QuantumCircuit
import sys
sys.path.append('utils')
import encryption_algorithms as enc

# Import CHSH core functions (solved in Part 1)
from utils.chsh_core import *

# Import E91 helper functions (solved in Part 2)
from utils.e91_core import *

# Use a DIFFERENT seed for this assignment!
GLOBAL_SEED = 42  # Different from workshop (91)
random.seed(GLOBAL_SEED)
np.random.seed(GLOBAL_SEED)

print(" Setup complete!")
print(f"\nAlice's bases: {ALICE_BASES}")
print(f"Bob's bases: {BOB_BASES}")

 Setup complete!

Alice's bases: ['0', '45', '90']
Bob's bases: ['45', '90', '135']


## Task 1: Create |Φ-⟩ Bell State

Complete the function to create |Φ-⟩ = (|00⟩ - |11⟩)/√2

**Hint:** Start with |Φ+⟩ and add ONE gate to create the minus sign!

```
|Φ+⟩: H → CX       gives (|00⟩ + |11⟩)/√2
|Φ-⟩: H → ? → CX   gives (|00⟩ - |11⟩)/√2
```

**Remember:** The Z gate adds a π phase: |1⟩ → -|1⟩


In [3]:
def create_bell_pair_phi_minus() -> QuantumCircuit:
    """
    TODO: Create the Bell state |Φ-⟩ = (|00⟩ - |11⟩)/√2

    Hint: Start with |Φ+⟩ (H + CX) and add ONE gate to create the minus sign!
    """
    qc = QuantumCircuit(2)

    # TODO: Add your gates here
    qc.h(0)
    qc.cx(0,1)
    qc.z(0)


    return qc

In [4]:
# Test your circuit
phi_minus = create_bell_pair_phi_minus()
print("Your circuit for |Φ-⟩:")
print(phi_minus.draw())

Your circuit for |Φ-⟩:
     ┌───┐     ┌───┐
q_0: ┤ H ├──■──┤ Z ├
     └───┘┌─┴─┐└───┘
q_1: ─────┤ X ├─────
          └───┘     


### Test the |Φ-⟩ State

Make sure you created the correct Bell state.  
Tip: $1/\sqrt{2}$ is the same as $\sqrt{2}/2$ 😄


In [5]:
from qiskit.quantum_info import Statevector
Statevector(phi_minus).draw('latex')

<IPython.core.display.Latex object>


## Verify: Test Your |Φ-⟩ Circuit

Run this to check if your circuit is correct. You should see ~50% |00⟩ and ~50% |11⟩.


In [6]:
# Test the |Φ-⟩ Bell state
phi_minus = create_bell_pair_phi_minus()

# Verify with measurements
test_qc = phi_minus.copy()
test_qc.measure_all()
counts = run_circuit(test_qc, shots=1000)

print(f"\nMeasurement results: {counts}")
print("\n Expected: ~50% |00⟩ and ~50% |11⟩ (never |01⟩ or |10⟩)")
print("   This confirms |Φ-⟩ is CORRELATED : ")
print("   --> Alice and Bob get SAME results: Alice=0 → Bob=0, Alice=1 → Bob=1")

print("\n Note: The minus sign (phase) doesn't affect Z-basis measurements,")
print("   but it DOES affect measurements in rotated bases (used for CHSH)!")


Measurement results: {'11': 478, '00': 522}

 Expected: ~50% |00⟩ and ~50% |11⟩ (never |01⟩ or |10⟩)
   This confirms |Φ-⟩ is CORRELATED : 
   --> Alice and Bob get SAME results: Alice=0 → Bob=0, Alice=1 → Bob=1

 Note: The minus sign (phase) doesn't affect Z-basis measurements,
   but it DOES affect measurements in rotated bases (used for CHSH)!



## Task 2: Find the Correct CHSH Formula

The CHSH formula has ONE minus sign. For |Φ-⟩, it moves to a different position!

**Workshop formula (for |Ψ-⟩):**
$$S = E(a,b) - E(a,b') + E(a',b) + E(a',b')$$

**Your task:** Try all 4 positions for the minus sign and find which gives |S| ≈ 2√2 ≈ 2.83


**First we generate test data for |Φ-⟩ :**

In [7]:
# Generate test data for |Φ-⟩

print("Generating 500 measurements for CHSH test...\n")
test_num_pairs = 500

# Generate random bases
test_alice_chsh_bases = [random.choice(ALICE_CHSH_BASES) for _ in range(test_num_pairs)]
test_bob_chsh_bases = [random.choice(BOB_CHSH_BASES) for _ in range(test_num_pairs)]

# Generate Bell pairs and Measure them
results = []
for a, b in zip(test_alice_chsh_bases, test_bob_chsh_bases):
    qc = create_bell_pair_phi_minus()
    results.append(measure_bell_pair(qc, a, b))


# step 4: Organize measurements by basis to compute correkations:
measurements = organize_measurements_by_basis(results, test_alice_chsh_bases, test_bob_chsh_bases)
correlations = calculate_correlations(measurements)


# Step 5: TODO
# Based on the correlations, compute and find the correct CHSH formulat (in next cell)

Generating 500 measurements for CHSH test...

Correlations:
  E('0', '45') = 0.6183
  E('0', '135') = -0.6615
  E('90', '45') = -0.7234
  E('90', '135') = -0.8207


In [8]:
# ═══════════════════════════════════════════════════════════
# TODO: Try all 4 minus sign positions!
# ═══════════════════════════════════════════════════════════

a1, a2 = '0', '90'   # Alice's CHSH bases
b1, b2 = '45', '135' # Bob's CHSH bases

E_ab = correlations[(a1, b1)]              # E(0°, 45°)
E_ab_prime = correlations[(a1, b2)]        # E(0°, 135°)
E_a_prime_b = correlations[(a2, b1)]       # E(90°, 45°)
E_a_prime_b_prime = correlations[(a2, b2)] # E(90°, 135°)

print("="*60)
print("Testing all 4 CHSH formula variations:")
print("="*60)

# TODO: Calculate S for each position of the minus sign

# Position 1: minus at first term
S_minus_p1 = -E_ab + E_ab_prime + E_a_prime_b + E_a_prime_b_prime

# Position 2: minus at second term (workshop formula)
S_minus_p2 = E_ab - E_ab_prime + E_a_prime_b + E_a_prime_b_prime

# Position 3: minus at third term
S_minus_p3 = E_ab + E_ab_prime - E_a_prime_b + E_a_prime_b_prime

# Position 4: minus at fourth term
S_minus_p4 = E_ab + E_ab_prime + E_a_prime_b - E_a_prime_b_prime

print("\n" + "="*60)
print(f"Target: 2√2 ≈ {2*np.sqrt(2):.4f}")
print("The formula with |S| ≈ 2.83 is the CORRECT one!")
print("="*60)

print("S1 =", abs(S_minus_p1))
print("S2 =", abs(S_minus_p2))
print("S3 =", abs(S_minus_p3))
print("S4 =", abs(S_minus_p4))

Testing all 4 CHSH formula variations:

Target: 2√2 ≈ 2.8284
The formula with |S| ≈ 2.83 is the CORRECT one!
S1 = 2.823952982717047
S2 = 0.2642348382660784
S3 = 0.14050325070470338
S4 = 0.0540675490018262


### Your Answer: Which CHSH formula is correct?

Based on your results above, write down the correct CHSH formula for |Φ-⟩:


In [9]:
def calculate_chsh_value_phi_minus(correlations, alice_bases=ALICE_CHSH_BASES, bob_bases=BOB_CHSH_BASES):
    """
    TODO: Implement the correct CHSH formula for |Φ-⟩

    Based on your experiment above, which position gives |S| ≈ 2√2?
    """
    a1, a2 = alice_bases  # '0', '90'
    b1, b2 = bob_bases    # '45', '135'

    # TODO: Write the correct formula here
    # S = ???
    S = (
    -correlations[(a1, b1)]
    + correlations[(a1, b2)]
    + correlations[(a2, b1)]
    + correlations[(a2, b2)]
)


    return abs(S)


# Test your formula
chsh = calculate_chsh_value_phi_minus(correlations)
print(f"Your CHSH value: {chsh:.4f}")
print(f"Expected: ≈ {2*np.sqrt(2):.4f}")

Your CHSH value: 2.8240
Expected: ≈ 2.8284


---
## E91 Protocol Helper Functions

These functions orchestrate the full E91 key distribution protocol.


In [10]:
# ============================================================
# E91 Wrappers — same logic as Part 2, but using YOUR functions
# ============================================================
# These are thin wrappers that call YOUR student-defined functions:
#   - create_bell_pair_phi_minus()        (Task 1)
#   - calculate_chsh_value_phi_minus()    (Task 2)
# All other helpers (measure_all_pairs, extract_e91_key_and_bell_test_data, etc.)
# are already imported from utils/e91_core.py

def create_list_bell_pairs(num_pairs):
    """Create a list of |Φ-⟩ Bell pairs (using YOUR function from Task 1)."""
    return [create_bell_pair_phi_minus() for _ in range(num_pairs)]


def check_for_eavesdropping(chsh_results, chsh_alice_bases, chsh_bob_bases):
    """Run CHSH security test (using YOUR formula from Task 2)."""
    bell_results = organize_measurements_by_basis(chsh_results, chsh_alice_bases, chsh_bob_bases)
    correlations = calculate_correlations(bell_results)
    chsh_value = calculate_chsh_value_phi_minus(correlations)
    return {
        'chsh_value': chsh_value,
        'is_secure': chsh_value > BELL_INEQUALITY_THRESHOLD
    }

print(" E91 wrappers loaded (using your Task 1 & Task 2 functions!)")

 E91 wrappers loaded (using your Task 1 & Task 2 functions!)



## E91 Protocol with |Φ-⟩ (Provided)

The `run_e91_protocol` function below puts everything together.

> **Note:** Key extraction for |Φ-⟩ is different from |Ψ-⟩!
> - |Ψ-⟩ (singlet) is anti-correlated in **every** basis → Bob always flips
> - |Φ-⟩ is **correlated in Z-basis (0°)** but **anti-correlated in X-basis (90°)**, and uncorrelated at 45°
> - Therefore, only **(90°, 90°)** pairs are used for key generation, and Bob flips his bits



In [11]:
# ============================================================
# E91 PROTOCOL with |Φ-⟩ (corrected with Bob flip)
# ============================================================

# Reset seed for reproducibility
MANUAL_SIMULATOR_SEED_COUNTER = 42
random.seed(GLOBAL_SEED)
np.random.seed(GLOBAL_SEED)

def run_e91_protocol(num_pairs=800):
    """
    Run E91 protocol with |Φ-⟩ Bell state.

    - |Φ-⟩ is correlated in Z-basis
    - ANTI-correlated in X-basis (90°)
    - Using (90°,90°) for key generation
    - Bob must flip his bits
    """
    print("=" * 60)
    print("E91 PROTOCOL with |Φ-⟩ Bell State")
    print("=" * 60)

    # Step 1: Generate Bell pairs
    print(f"\n1. Generating {num_pairs} |Φ-⟩ Bell pairs...")
    bell_pairs = create_list_bell_pairs(num_pairs)

    # Step 2: Generate random bases
    print("\n2. Alice and Bob choose random bases...")
    alice_bases = generate_random_bases(num_pairs, ALICE_BASES)
    bob_bases = generate_random_bases(num_pairs, BOB_BASES)

    # Step 3: Measure all pairs
    print("\n3. Measuring all pairs...")
    results = measure_all_pairs(bell_pairs, alice_bases, bob_bases)

    # Step 4: Sifting
    print("\n4. Sifting results...")
    key_results = []
    chsh_results = []
    chsh_alice_bases = []
    chsh_bob_bases = []

    for result, a_base, b_base in zip(results, alice_bases, bob_bases):
        if a_base == '90' and b_base == '90':
            key_results.append(result)
        elif (a_base, b_base) in CHSH_BASIS_PAIRS:
            chsh_results.append(result)
            chsh_alice_bases.append(a_base)
            chsh_bob_bases.append(b_base)

    print(f"   Key generation pairs (90°,90°): {len(key_results)}")
    print(f"   CHSH test pairs: {len(chsh_results)}")

    # Step 5: Security check
    print("\n5. Running CHSH security test...")
    security = check_for_eavesdropping(chsh_results, chsh_alice_bases, chsh_bob_bases)

    print(f"\n   CHSH Value: {security['chsh_value']:.4f}")
    print(f"   Classical limit: 2.0")
    print(f"   Quantum limit: 2√2 ≈ 2.83")

    if not security['is_secure']:
        print("\n SECURITY CHECK FAILED!")
        print("   Possible eavesdropping detected.")
        return None

    print("\n SECURITY CHECK PASSED!")

    # Step 6: Extract key
    print("\n6. Extracting shared key...")

    # Qiskit format: result = "BA"
    alice_key = ''.join([r[0] for r in key_results])
    bob_key   = ''.join([r[1] for r in key_results])

    if alice_key == bob_key:
        print("\n Keys match perfectly!")
    else:
        mismatches = sum(a != b for a, b in zip(alice_key, bob_key))
        print(f"\n {mismatches} mismatches in {len(alice_key)} bits")

    print(f"   Key length: {len(alice_key)} bits")
    print(f"   Key (first 50): {alice_key[:50]}...")

    return bob_key


print(" E91 Protocol function loaded!")

 E91 Protocol function loaded!


### Run E91 protocol

In [12]:
key = run_e91_protocol(num_pairs=800)

E91 PROTOCOL with |Φ-⟩ Bell State

1. Generating 800 |Φ-⟩ Bell pairs...

2. Alice and Bob choose random bases...

3. Measuring all pairs...

4. Sifting results...
   Key generation pairs (90°,90°): 89
   CHSH test pairs: 362

5. Running CHSH security test...
Correlations:
  E('0', '45') = 0.7474
  E('0', '135') = -0.6591
  E('90', '45') = -0.6941
  E('90', '135') = -0.7234

   CHSH Value: 2.8240
   Classical limit: 2.0
   Quantum limit: 2√2 ≈ 2.83

 SECURITY CHECK PASSED!

6. Extracting shared key...

 89 mismatches in 89 bits
   Key length: 89 bits
   Key (first 50): 01110101101011110100011111111111100100001001011001...



### Decrypt the Secret Messages

If your protocol worked correctly, you should be able to decrypt the messages!


In [13]:
def decrypt_and_print_messages(key: str, filename: str = "encrypted_messages.txt"):
    """
    Read encrypted messages from a file, decrypt them using XOR with the given key, and print.

    Args:
        key (str): The key to use for XOR decryption.
        filename (str): The file to read encrypted messages from.
    """
    print("\nDecrypting all messages from", filename)
    with open(filename, "r") as f:
        for line in f:
            if ": " in line:
                msg_id, encrypted = line.split(": ", 1)
                decrypted = enc.decrypt_xor_repeating_key(encrypted.strip(), key)
                print(f"{msg_id}: {decrypted}")


In [14]:
# For your challenge: must generate the correct key using the E91 protocol to decrypt the messages. Without the key, decryption is not feasible.
# change the path to the encrypted file as needed
path_to_encrypted_file = R"assignment_encrypted_messages.txt"
decrypt_and_print_messages(key, filename=path_to_encrypted_file)



Decrypting all messages from assignment_encrypted_messages.txt
Message 1: "Science knows no country, because knowledge belongs to humanity."  # Louis Pasteur
Message 2: "Mathematics is the queen of the sciences and number theory is the queen of mathematics."  # Carl Friedrich Gauss



## Summary: What You Learned

Complete this table based on your work:

| Aspect             | Workshop ($\vert \Psi^- \rangle$)         | This Assignment ($\vert \Phi^- \rangle$) |
|-------------------|-------------------------------------------|-----------------------------------------|
| Circuit            | H → CX → X → Z                             | H → ??? → CX                            |
| Correlation        | Anti-correlated                             | ???                                     |
| Key extraction     | Bob flips bits                              | ???                                     |
| CHSH formula       | $S = +E(a,b) - E(a,b') + E(a',b) + E(a',b')$ | $S = ???$                              |

**Key insight:** Any Bell state can be used for E91, but you need to:  
1. Know if it's **correlated** or **anti-correlated** (for key extraction)  
2. Find the **correct CHSH formula** (the one that gives $|S| \approx 2\sqrt{2}$)
